In [49]:
import nltk
#nltk.download('stopwords')
nltk.download('wordnet')
from sagemaker.tensorflow import TensorFlow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import unicodedata

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [50]:
import boto3
s3=boto3.resource("s3")
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping,ModelCheckpoint
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn.manifold import TSNE


In [51]:
review = pd.read_csv("new_dataset_indian_resto_review.csv")

In [52]:
def find_useful(df):
#     if (df["useful"] == 0) & (df["funny"] == 0) & (df["cool"] == 0):
#         return 0
#     else:
#         return 1
    
    #print(df[4],df[7],df[8])
    if df[4] == 0 and df[7]==0 and df[8] ==0:
        return 0
    else:
        return 1
        
#         return df[4]
#     else:
#         return 1
    


In [53]:
review["label"] = review.apply(find_useful,axis=1)

In [54]:
review.head(2)

,Unnamed: 0,business_id,name,categories,useful,text,stars,funny,cool,label
0,0,iE71iwcSljg3xm2GB2Y9aA,Marigold Maison- Paradise Valley,"Indian, Restaurants",0,I have been here several times since my last u...,5.0,0,0,0
1,1,DJuQgsoj-qmU0i5iTAa4aA,Tadka Restaurant,"Restaurants, Indian",0,We just tried to eat here and I'm pretty sure ...,1.0,1,0,1


In [55]:
review = review[["text","label"]]

In [61]:
review = review.dropna()
review.head(5)
review["label"].value_counts()

1    43195
0    36597
Name: label, dtype: int64

In [62]:
review.head(2)

,text,label,normalizes_text,remove_special,token_text,remove_stop,lemitize_text
0,I have been here several times since my last u...,0,I have been here several times since my last u...,I have been here several times since my last u...,"[i, have, been, here, several, times, since, m...","[several, times, since, last, update, , , food...",several time since last update food always t...
1,We just tried to eat here and I'm pretty sure ...,1,We just tried to eat here and I'm pretty sure ...,We just tried to eat here and Im pretty sure i...,"[we, just, tried, to, eat, here, and, im, pret...","[tried, eat, im, pretty, sure, closed, , table...",tried eat im pretty sure closed table pushed ...


In [63]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode()
stopwords_ = set(stopwords.words("english"))
lemmatizer=WordNetLemmatizer()

In [64]:
def pre_processing(india_review):
    india_review["normalizes_text"] = india_review["text"].apply(lambda x:remove_accents(x))
    #india_review["remove_html"] = india_review['normalizes_text'].apply(lambda x: strip_html_tags(x))
    india_review["remove_special"] = india_review["normalizes_text"].replace(r'[^A-Za-z0-9 ]+', '', regex=True)
    
    india_review["token_text"] = india_review["remove_special"].apply(lambda x:[word.lower() for word in x.split(" ")]) 
    #india_review["token_text"] = india_review["token_text"].apply(lambda x:[word for word in x if word not in common_words_list])
    
    india_review["remove_stop"] = india_review["token_text"].apply(lambda x:[word for word in x if word not  in stopwords_])
    india_review["lemitize_text"] = india_review["remove_stop"].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x]))
 #   india_review["length"] = india_review["remove_stop"].apply(lambda x:find_length(x))
   # review["clean_list"] = review["lemitize_text"].apply(lambda x:[word for word in x.split(" ")])
#    india_review["texts_to_sequences"] = india_review["lemitize_text"].apply(lambda x:)

    return india_review
     
  
    

In [72]:
review=pre_processing(review)

In [94]:
#review["lemitize_text"]
train_set = review.sample(frac=0.75, random_state=0)
test_set = review.drop(train_set.index)

In [68]:
train_review = train_set["lemitize_text"]
test_review = test_set["lemitize_text"]
train_review.head()
label_train = train_set["label"]
label_test = test_set["label"]



In [92]:
type(train_review)

pandas.core.series.Series

In [75]:
train_review.head(5)

42791    went lunch service ok napkin paper towel bathr...
33833    came excellent yelp review let say yelp commun...
40033    place attached holiday inn let fool food amazi...
36423    making visit namaste never tried indian partic...
28831    place delicious dinner family 7 staff friendly...
Name: lemitize_text, dtype: object

In [81]:
def token_sequn(review):
    max_len=50
    embed_dim=100
    max_words=20000
    tokenizer = Tokenizer(num_words=max_words,char_level=True)
    tokenizer.fit_on_texts(review["lemitize_text"])
    sequences = tokenizer.texts_to_sequences(review["lemitize_text"])
    data = pad_sequences(sequences,maxlen=max_len,padding='post')
    

In [122]:
len(data[0])

50

In [124]:
labels = np.asarray(review["label"])